# ONTOLOGY EXPLORATION

In [ ]:
import pathlib
import sys

NOTEBOOK_DIR = pathlib.Path.cwd()
if '_NB_SYS_PATH_ADJUSTED' not in globals():
    sys.path.insert(0, str(NOTEBOOK_DIR))
    _NB_SYS_PATH_ADJUSTED = True

import pandas as pd 
import rdflib

import modules.file_utils as file_utils

pd.set_option('display.max_colwidth', None)

In [ ]:
import modules.rdf_utils as rdf_utils

def load_graph(input_ttl: pathlib.Path) -> rdflib.Graph:
    graph = rdf_utils.read_graph(str(input_ttl))
    return graph


In [ ]:
#  read input ontology file
INPUT_TTL = file_utils.latest_file(folder=NOTEBOOK_DIR / "ontologies", filename_fragment="ontology_")
g = load_graph(INPUT_TTL)


## QUERIES

In [ ]:
import modules.run_query as run_query

In [ ]:
CONCEPTS_QUERY = """\
SELECT DISTINCT
?concept_iri
?concept_label
WHERE {
    ?concept_iri a core:concept .
    OPTIONAL { ?concept_iri rdfs:label ?concept_label . }
}
ORDER BY ?concept_iri
"""
concepts_df = run_query.main(g, CONCEPTS_QUERY, write_to_csv=True)
concepts_df

In [ ]:
POSTULATES_QUERY = """\
SELECT DISTINCT
?postulate_iri
WHERE {
    ?postulate_iri a core:postulate .
}       
ORDER BY ?postulate_iri
"""
postulates_df = run_query.main(g, POSTULATES_QUERY, write_to_csv=False)
postulates_df

In [ ]:
COMMON_NOTIONS_QUERY = """\
SELECT DISTINCT
?common_notion_iri
WHERE {
    ?common_notion_iri a core:common_notion .
}       
ORDER BY ?common_notion_iri
"""
common_notion_df = run_query.main(g, COMMON_NOTIONS_QUERY, write_to_csv=False)
common_notion_df

In [ ]:
COMMON_NOTIONS_QUERY = """\
SELECT DISTINCT
    ?s ?o
WHERE {
    ?s owl:sameAs ?o .
}       
ORDER BY ?s ?o
"""
common_notion_df = run_query.main(g, COMMON_NOTIONS_QUERY, write_to_csv=False)
common_notion_df

In [ ]:
NEIGHBORHOOD_QUERY = """\
SELECT DISTINCT
    ?o
WHERE {
    ?s
        a core:concept ;
        core:is_concept_in ?o .
}
ORDER BY ?o
"""
neighborhood_df = run_query.main(g, NEIGHBORHOOD_QUERY, write_to_csv=False)
neighborhood_df

In [ ]:
NEIGHBORHOOD_COUNT_QUERY = """\
PREFIX core: <https://www.foom.com/core#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT
    ?o
    (COUNT(*) AS ?links)
WHERE {
    ?s
        a core:concept ;
        core:is_concept_in ?o .
}
GROUP BY ?o
ORDER BY DESC(?links)
"""
results = g.query(NEIGHBORHOOD_COUNT_QUERY)
for row in results:
    print(f"{row.links} {row.o}")
# neighborhood_df = run_query.main(g, NEIGHBORHOOD_QUERY, write_to_csv=False)
# neighborhood_df